In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tq
import time
from bs4 import BeautifulSoup
from requests import get
import pymongo
import re

In [2]:
client = pymongo.MongoClient('mongodb://127.0.0.1:27017/')
db = client.ad_database
url_collection = db.ad_urls

In [ ]:
def new_url_scrape():
    for i in tq(range(5884,6614)):
        url = 'http://www.welovead.com/en/works/basic_search/?page='+str(i)
        try:
            r = get(url)
        except:
            sleep(2)
            r = get(url)
        soup = BeautifulSoup(r.text, 'html.parser')
        body = soup.find('ul', id='ul_works_list').find_all('a')
        urls = []
        for x in body:
            url_dict = {}
            url = 'http://www.welovead.com' + x['href']
            url_dict['url'] = url
            urls.append(url_dict)
        url_collection.insert_many(urls)
    return None

In [ ]:
new_url_scrape()

In [3]:
urls = []
for x in url_collection.find({}):
    urls.append(x['url'])

In [4]:
urls = list(set(urls))
len(urls)

146856

In [5]:
urls_2 = urls[36714:73428]
len(urls_2)

36714

In [6]:
new_ad_collection = db.new_ad_collection

In [7]:
def big_scrape(urls):
    for url in tq(urls):
        ad_dict = {}
        try:
            r = get(url)
        except:
            time.sleep(1.5)
            r = get(url)
        soup = BeautifulSoup(r.text, 'html.parser')
        try:
            ad_dict['industry'] = soup.find('span', class_='subscribeTo').text.strip()
        except:
            ad_dict['indsutry'] = np.nan
        try:
            ad_dict['medium'] = soup.find('a',href = re.compile('\?m=\d{1,2}')).text
        except:
            ad_dict['medium'] = np.nan
        try:
            ad_dict['headline'] = soup.find('td', id='td_headline_copy_text').text.strip().replace('\r', '').replace('\n', ' ').replace('\t', ' ')
        except:
            ad_dict['headline'] = np.nan
        try:
            ad_dict['description'] = soup.find('td', id='td_short_description').text.strip().replace('\r', '').replace('\n', ' ').replace('\t', ' ')
        except:
            ad_dict['description'] = np.nan
        try:
            ad_dict['brand'] = soup.find('a', class_='iconBrand').text
        except:
            ad_dict['brand'] = np.nan
        ad_dict['url'] = url
        try:
            ad_dict['country'] = soup.find('a',href = re.compile('\?r=\d{1,3}')).text
        except:
            ad_dict['country'] = np.nan
        try:
            ad_dict['img_url'] = soup.find('div', class_='works_box').a['href']
        except:
            ad_dict['img_url'] = np.nan
        new_ad_collection.insert_one(ad_dict)
    return None

In [14]:
big_scrape(urls_2)

ConnectionError: HTTPConnectionPool(host='www.welovead.com', port=80): Max retries exceeded with url: /en/works/details/566wguvBf (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001C1CA23B198>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [8]:
urls_2_up = urls_2[12118:]
len(urls_2_up)

24596

In [ ]:
big_scrape(urls_2_up)

In [9]:
big_scrape(urls_2_up[16817:]) 